## Preprocessing 01 (Working with Sampling)
#### 1. Aquisition with Crawling
#### 2. Tranformation included Cleansing & Denoising
#### 3. Imputation

In [ ]:
import sys
import ssl
from urllib.request import Request, urlopen
from datetime import datetime

def crawling(url='', encoding='utf-8', err=lambda e: print(f'{e} : {datetime.now()}', file=sys.stderr)):
    try:
        req = Request(url)
        ssl._create_default_https_context = ssl._create_unverified_context

        resp = urlopen(req)
        recv = resp.read().decode(encoding, errors='replace')

        print(f'{datetime.now()}: success for request [{url}]')
        return recv
    except Exception as e:
        err(e)

In [ ]:
from functools import reduce
from itertools import count

resfetch = []
for idx in count(start=1):
    l = len(resfetch)
    resfetch[l:] = list(filter(
        lambda s: s.startswith('#'),
        reduce(
            lambda param, func: func(param),
            [lambda txt: [] if txt is None else txt.splitlines(), lambda strs: [f(s) for f in [str.strip] for s in strs]],
            crawling(
                'https://raw.githubusercontent.com/kickscar/HELLCHANG-PRACTICES/main/training-record/Week0%02d.md' % idx,
                err=lambda e: None
            )
        )
    ))
    if len(resfetch) == l:
        break

print(resfetch)

In [ ]:
import re

restransform = []
for res in resfetch:
    if res.startswith('####'):
        # 1. split name and record
        name, record = (s.strip() for s in res.replace('####', '').strip().split(':'))

        # 2. cleansing name
        name = re.sub(r'[\d+\\.]', '', name).strip()
        p = re.compile(r'([a-zA-Z\s\',]+)\s*\[([a-zA-Z\s\',]+)\]').match(name)
        if p is not None:
            name, tool = p.groups()
            name = ''.join([name.strip(), f'[{tool.strip()}]']).title()
        # sval, unit = re.compile(r'(\d+.\d*)\s*(.*)').match(re.sub(r"[\([{})\]\s]", "", val)).groups()

        restransform[len(restransform):] = [(date, name, *re.split(r'\s+', record.strip()))]
    elif res.startswith('##'):
        date = datetime.strptime(res.replace('##', '').strip(), '%Y/%m/%d')

print(restransform)

In [ ]:
# Which Exercise?

exs = [t[1] for t in restransform]
dataset_exs = [(ex, exs.count(ex)) for ex in list(set(exs))]

sorted_by_ex = sorted(dataset_exs, key=lambda t: t[0])
for d in sorted_by_ex:
    print(d)

In [ ]:
# and How Often?
sorted_by_count = sorted(dataset_exs, key=lambda t: t[1], reverse=True)
for d in sorted_by_count:
    print(d)